In [9]:
import os
import pandas as pd
from flask import Flask, render_template_string
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.font_manager import FontProperties
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

app = Flask(__name__)

@app.route('/')
def index():
    # SQL Server 連接變量
    sql_host = 'localhost'
    sql_dbname = 'StockData'
    table_name = 'StockPrices'

    # 構建連接字符串
    con_string = f"mssql+pyodbc://{sql_host}/{sql_dbname}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"
    engine = create_engine(con_string)

    # 從 SQL Server 讀取數據進入 DataFrame
    query = f"SELECT * FROM {table_name} ORDER BY Date"
    df = pd.read_sql(query, engine)

    # 將數據轉換為列表形式
    data = df.values.tolist()

    # 繪製圖表並保存為臨時文件
    font_path = 'C:/WINDOWS/FONTS/SIMSUN.TTC'  # 需要根據實際情況修改
    font = FontProperties(fname=font_path)
    
    plt.figure(figsize=(12, 6))
    plt.plot(df['Date'], df['ClosePrice'], label='Close Price')
    plt.plot(df['Date'], df['LowPrice'], label='Low Price')
    plt.plot(df['Date'], df['HighPrice'], label='High Price')
    plt.title('Stock Price Trends for 0050', fontproperties=font)
    plt.xlabel('Date', fontproperties=font)
    plt.ylabel('Price (NTD)', fontproperties=font)
    plt.legend()
    plt.grid(True)
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m'))

    # 確保 static 目錄存在
    if not os.path.exists('static'):
        os.makedirs('static')

    temp_img_path = os.path.join('static', 'stock_trend.png')
    plt.savefig(temp_img_path)
    plt.close()

    # 構造 HTML 內容
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>股票數據</title>
    </head>
    <body>
        <h1>股票數據</h1>
        <table border="1">
            <tr>
                <th>Date</th>
                <th>ClosePrice</th>
                <th>LowPrice</th>
                <th>HighPrice</th>
            </tr>
            {% for row in data %}
            <tr>
                <td>{{ row[0] }}</td>
                <td>{{ row[1] }}</td>
                <td>{{ row[2] }}</td>
                <td>{{ row[3] }}</td>
            </tr>
            {% endfor %}
        </table>
        <br>
        <img src="{{ chart_path }}" alt="股票趨勢圖">
    </body>
    </html>
    """

    # 渲染 HTML 內容並傳遞數據和圖表路徑
    return render_template_string(html_content, data=data, chart_path=temp_img_path)

def fetch_and_store_data():
    # 設置 Chrome 選項
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    
    # 初始化 Chrome WebDriver
    driver = webdriver.Chrome(options=chrome_options)
    
    # 抓取股票數據函數
    def fetch_stock_data(year, month, stock_number='0050'):
        url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date={year}{month:02d}01&stockNo={stock_number}'
        df = pd.read_html(url, header=0)[0]

        # 打印數據前幾行進行調試
        print("原始數據：")
        print(df.head())

        # 重命名列
        df.columns = ['Date', 'TradeVolume', 'TradeAmount', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'PriceDiff', 'TradeCount']

        # 刪除不需要的行（如標題行和總計行）
        df = df[df['Date'] != '日期']
        df = df.dropna(subset=['Date'])
        
        # 只保留需要的列：Date, ClosePrice, LowPrice, HighPrice
        df = df[['Date', 'ClosePrice', 'LowPrice', 'HighPrice']]

        # 轉換日期格式
        df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: f"{int(x.split('/')[0]) + 1911}/{x.split('/')[1]}/{x.split('/')[2]}"), format='%Y/%m/%d')

        # 轉換價格為浮點數
        df['ClosePrice'] = df['ClosePrice'].astype(float)
        df['LowPrice'] = df['LowPrice'].astype(float)
        df['HighPrice'] = df['HighPrice'].astype(float)
        
        print("處理後數據：")
        print(df.head())
        
        return df

    # 準備一整年的數據並存儲到數據庫和CSV文件
    def prepare_full_year_data(stock_number='0050'):
        df_all = pd.DataFrame()
        for month in range(1, 13):
            df_month = fetch_stock_data(2023, month, stock_number)
            df_all = pd.concat([df_all, df_month], ignore_index=True)
            time.sleep(2)  # 暫停2秒以避免過快訪問網站
        return df_all

    df_year = prepare_full_year_data(stock_number='0050')

    # 保存 DataFrame 為 Excel 文件
    current_dir = os.getcwd()
    file_path = os.path.join(current_dir, 'output.xlsx')
    df_year.to_excel(file_path, index=False)
    print(f"數據已成功保存到 {file_path}")
    
    # SQL Server 連接變量
    sql_host = 'localhost'
    sql_dbname = 'StockData'
    
    # 構建連接字符串
    con_string = f"mssql+pyodbc://{sql_host}/{sql_dbname}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"
    engine = create_engine(con_string)
    
    # 將 DataFrame 保存到 SQL Server
    df_year.to_sql('StockPrices', engine, if_exists='replace', index=False)
    print("數據已成功保存到 SQL Server")

if __name__ == "__main__":
    fetch_and_store_data()
    app.run(debug=True, use_reloader=False)


原始數據：
  112年01月 0050 元大台灣50 各日成交資訊 112年01月 0050 元大台灣50 各日成交資訊.1  \
0                         日期                         成交股數   
1                  112/01/03                     15272877   
2                  112/01/04                     14211742   
3                  112/01/05                     12395862   
4                  112/01/06                     11826560   

  112年01月 0050 元大台灣50 各日成交資訊.2 112年01月 0050 元大台灣50 各日成交資訊.3  \
0                         成交金額                          開盤價   
1                   1673263794                       109.60   
2                   1569704151                       110.30   
3                   1379134974                       111.25   
4                   1321964557                       111.20   

  112年01月 0050 元大台灣50 各日成交資訊.4 112年01月 0050 元大台灣50 各日成交資訊.5  \
0                          最高價                          最低價   
1                       110.90                       108.45   
2                       110.80                       110.10

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Jul/2024 19:34:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2024 19:34:11] "GET /static/stock_trend.png HTTP/1.1" 200 -
